# Deep Learning

## preprocessing training dataset

In [1]:
import os
import numpy as np
import tensorflow as tf
import pandas as pd
input_file_dir = "/Users/ninglee/Documents/IntrutionDection/datasets"
train_file_name = "kddcup.data_10_percent.txt"
test_file_name = "corrected.txt"
header_file_name = "header.txt"
train_files = os.path.join(input_file_dir, train_file_name)
test_files = os.path.join(input_file_dir, test_file_name)
header_files = os.path.join(input_file_dir, header_file_name)

In [2]:
with open(header_files, 'r') as f:
    header = f.readline().strip().split(',')
train_dataset = pd.read_csv(train_files)
test_dataset = pd.read_csv(test_files)
train_dataset.columns = header
test_dataset.columns = header

In [3]:
train_dataset_size = train_dataset.shape[0]
test_dataset_size = test_dataset.shape[0]
train_dataset = pd.concat([train_dataset, test_dataset], axis=0)
print train_dataset_size, test_dataset_size

494020 311028


In [4]:
def labels_map(label):
    label = str(label).split('.')[0]
    if label == 'normal':
        return 0
    if label in ['ipsweep', 'mscan', 'nmap', 'portsweep', 'saint', 'satan']: #PROBE
        return 1
    if label in ['apache2', 'back', 'land', 'mailbomb', 'neptune', 'pod', 'processtable', 'smurf', 'teardrop', 'udpstorm']: #DOS
        return 2
    if label in ['buffer_overflow', 'httptunnel', 'loadmodule', 'perl', 'ps', 'rootkit', 'sqlattack', 'xterm']: #U2R
        return 3
    if label in ['ftp_write', 'guess_passwd', 'imap', 'multihop', 'named', 'phf', 'sendmail', 'snmpgetattack', 'snmpguess', 'spy', 'warezclient', 'warezmaster', 'worm', 'xlock', 'xsnoop']: #R2L
        return 4
    
train_dataset['labels'] = train_dataset['labels'].apply(labels_map)
labels_dummies = pd.get_dummies(train_dataset['labels'], prefix='label')
train_dataset = pd.concat([train_dataset,labels_dummies], axis=1)

In [5]:
train_dataset.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,labels,label_0,label_1,label_2,label_3,label_4
0,0,tcp,http,SF,239,486,0,0,0,0,...,0.0,0.0,0.0,0.0,0,1,0,0,0,0
1,0,tcp,http,SF,235,1337,0,0,0,0,...,0.0,0.0,0.0,0.0,0,1,0,0,0,0
2,0,tcp,http,SF,219,1337,0,0,0,0,...,0.0,0.0,0.0,0.0,0,1,0,0,0,0
3,0,tcp,http,SF,217,2032,0,0,0,0,...,0.0,0.0,0.0,0.0,0,1,0,0,0,0
4,0,tcp,http,SF,217,2032,0,0,0,0,...,0.0,0.0,0.0,0.0,0,1,0,0,0,0


In [6]:
protocal_type_dummies = pd.get_dummies(train_dataset.protocol_type, prefix='protocol_type')
service_dummies = pd.get_dummies(train_dataset.service, prefix='service')
flag_dummies = pd.get_dummies(train_dataset.flag, prefix='flag')
train_dataset = pd.concat([train_dataset, protocal_type_dummies, service_dummies, flag_dummies], axis=1)

In [7]:
max1 = train_dataset.src_bytes.max(); min1 = train_dataset.src_bytes.min();
max2 = train_dataset.dst_bytes.max(); min2 = train_dataset.dst_bytes.min();
train_dataset['src_bytes_norm'] = (train_dataset.src_bytes - min1) / float(max1 - min1)
train_dataset['dst_bytes_norm'] = (train_dataset.dst_bytes - min2) / float(max2 - min2)

In [8]:
train_dataset = train_dataset.drop(['protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes','labels'], axis=1)

In [9]:
train_dataset = train_dataset.astype('float')
# train_dataset = (train_dataset - train_dataset.min()) / (train_dataset.max() - train_dataset.min())

## preprocessing test dataset

In [10]:
sub_train_dataset = train_dataset.iloc[train_dataset_size:, :].sample(n=200000)
sub_test_dataset = train_dataset.iloc[:train_dataset_size, :].sample(n=50000)
sub_train_labels = sub_train_dataset[['label_0', 'label_1', 'label_2', 'label_3', 'label_4']]
sub_test_labels = sub_test_dataset[['label_0', 'label_1', 'label_2', 'label_3', 'label_4']]
sub_train_dataset.drop(['label_0', 'label_1', 'label_2', 'label_3', 'label_4'], axis=1, inplace=True)
sub_test_dataset.drop(['label_0', 'label_1', 'label_2', 'label_3', 'label_4'], axis=1, inplace=True)

In [11]:
test_dataset = train_dataset.iloc[train_dataset_size:,:]
train_dataset = train_dataset.iloc[:train_dataset_size, :]
train_labels = train_dataset[['label_0', 'label_1', 'label_2', 'label_3', 'label_4']]
test_labels = test_dataset[['label_0', 'label_1', 'label_2', 'label_3', 'label_4']]
train_dataset.drop(['label_0', 'label_1', 'label_2', 'label_3', 'label_4'], axis=1, inplace=True)
test_dataset.drop(['label_0', 'label_1', 'label_2', 'label_3', 'label_4'], axis=1, inplace=True)

In [12]:
print train_dataset.shape, train_labels.shape
print test_dataset.shape, test_labels.shape

(494020, 119) (494020, 5)
(311028, 119) (311028, 5)


In [13]:
print sub_train_dataset.shape, sub_train_labels.shape
print sub_test_dataset.shape, sub_test_labels.shape

(200000, 119) (200000, 5)
(50000, 119) (50000, 5)


In [14]:
sub_train_dataset.describe()

,duration,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,...,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH,src_bytes_norm,dst_bytes_norm
count,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,...,200000.000000,200000.000000,200000.000000,200000.0000,200000.000000,200000.000000,200000.000000,200000.00000,2.000000e+05,200000.000000
mean,17.169845,0.000020,0.000800,0.000040,0.014955,0.002455,0.171790,0.010575,0.000205,0.000025,...,0.000005,0.002790,0.058225,0.0001,0.000050,0.000930,0.798005,0.00024,2.290441e-06,0.000146
std,377.184612,0.004472,0.041223,0.007746,0.265088,0.050784,0.377199,1.662136,0.014316,0.005916,...,0.002236,0.052747,0.234169,0.0100,0.007071,0.030482,0.401490,0.01549,1.065741e-04,0.003359
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.00000,0.000000e+00,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,1.000000,0.00000,1.514331e-07,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,1.000000,0.00000,7.499542e-07,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,1.000000,0.00000,1.488371e-06,0.000000
max,53771.000000,1.000000,3.000000,2.000000,30.000000,4.000000,1.000000,611.000000,1.000000,2.000000,...,1.000000,1.000000,1.000000,1.0000,1.000000,1.000000,1.000000,1.00000,4.563992e-02,1.000000


## model1: MLP

In [15]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [16]:
batch_size = 1000
feature_size = 119
num_labels = 5

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, feature_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
#   tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(sub_test_dataset.values[:,:])
  
  # Variables.
  hidden_weights = tf.Variable(
    tf.truncated_normal([feature_size, 1024]))
  output_weights = tf.Variable(
    tf.truncated_normal([1024, num_labels]))
  weights = [hidden_weights, output_weights]
  hidden_biases = tf.Variable(tf.zeros([1024]))
  output_biases = tf.Variable(tf.zeros([num_labels]))
  biases = [hidden_biases, output_biases]
  
  # Training computation.
  hidden_layer = tf.add(tf.matmul(tf_train_dataset, tf.cast(weights[0], tf.float32)), biases[0])
  hidden_layer = tf.nn.relu(hidden_layer)
  logits = tf.add(tf.matmul(hidden_layer, tf.cast(weights[1], tf.float32)), biases[1])
#   logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
  l2Loss = tf.nn.l2_loss(weights[0]) + tf.nn.l2_loss(weights[1])
  loss = loss + 5*l2Loss
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.001).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
    
  # ValidPrediction
#   temp1 = tf.add(tf.matmul(tf_valid_dataset, weights[0]), biases[0])
#   hidden_temp = tf.nn.relu(temp1)
#   temp2 = tf.add(tf.matmul(hidden_temp,weights[1]), biases[1])
#   valid_prediction = tf.nn.softmax(temp2)

  # TestPrediction
  temp3 = tf.add(tf.matmul(tf.cast(tf_test_dataset, tf.float32), weights[0]), biases[0])
  hidden_temp2 = tf.nn.relu(temp3)
  temp4 = tf.add(tf.matmul(tf.cast(hidden_temp2, tf.float32), weights[1]), biases[1])
  test_prediction = tf.nn.softmax(temp4)

In [17]:
num_steps = 6001

with tf.Session(graph=graph) as session:
  #tf.initialize_all_variables().run()
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (sub_train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = sub_train_dataset.iloc[offset:(offset + batch_size), :].values[:,:]
    batch_labels = sub_train_labels.iloc[offset:(offset + batch_size), :].values[:,:]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 300 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
#       print("Validation accuracy: %.1f%%" % accuracy(
#         valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), sub_test_labels.values[:,:]))

Initialized
Minibatch loss at step 0: 253255.406250
Minibatch accuracy: 0.9%
Test accuracy: 79.5%
Minibatch loss at step 300: 12116.498047
Minibatch accuracy: 83.5%
Test accuracy: 96.7%
Minibatch loss at step 600: 596.876648
Minibatch accuracy: 87.8%
Test accuracy: 96.9%
Minibatch loss at step 900: 29.804335
Minibatch accuracy: 91.0%
Test accuracy: 95.4%
Minibatch loss at step 1200: 1.917058
Minibatch accuracy: 90.7%
Test accuracy: 95.3%
Minibatch loss at step 1500: 0.561843
Minibatch accuracy: 89.8%
Test accuracy: 95.3%
Minibatch loss at step 1800: 0.510866
Minibatch accuracy: 89.2%
Test accuracy: 95.3%
Minibatch loss at step 2100: 0.513499
Minibatch accuracy: 88.8%
Test accuracy: 95.3%
Minibatch loss at step 2400: 0.503703
Minibatch accuracy: 89.8%
Test accuracy: 95.4%
Minibatch loss at step 2700: 0.436318
Minibatch accuracy: 91.7%
Test accuracy: 95.3%
Minibatch loss at step 3000: 0.458622
Minibatch accuracy: 90.2%
Test accuracy: 95.3%
Minibatch loss at step 3300: 0.448924
Minibatch 